In [7]:
!pip install requests
!pip install beautifulsoup4

In [94]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse

def format_url(base_url, path):  
  if path.startswith('/') or path.startswith('./'):
    path = path.split('/', 1)[-1]
    return os.path.join(base_url, path)

  return path
      

def generate(url, url_lim, re_lim):
  r = requests.get(url)
  soup = BeautifulSoup(r.content, 'html.parser')

  links = soup.find_all('a')
  link_urls = [link.get('href') for link in links]
  # Format URLs
  link_urls = [format_url(url, link_url) for link_url in filter(None, link_urls)]
  # Remove duplidcate URLs
  link_urls = list(set(link_urls))

  for link_url in link_urls[:url_lim]:
    print(url + ' --> ' + link_url)

    # Recursive
    if re_lim > 0:
      generate(link_url, url_lim, re_lim-1)

generate('https://translate.google.co.id', 2, 2)

https://translate.google.co.id --> https://stadia.google.com/
https://stadia.google.com/ --> https://www.facebook.com/GoogleStadia/
https://www.facebook.com/GoogleStadia/ --> https://www.facebook.com/GoogleStadia/allactivity?privacy_source=activity_log_top_menu
https://www.facebook.com/GoogleStadia/ --> https://www.facebook.com/GoogleStadia/fundraisers/
https://stadia.google.com/ --> https://store.google.com/product/stadia
https://store.google.com/product/stadia --> https://store.google.com/product/stadia/orderhistory?hl=en-US
https://store.google.com/product/stadia --> https://store.google.com/product/stadia/product/stadia_reviews
https://translate.google.co.id --> https://ads.google.com/home/?subid=ww-ww-et-g-aw-a-vasquette_ads_cons_1!o2
https://ads.google.com/home/?subid=ww-ww-et-g-aw-a-vasquette_ads_cons_1!o2 --> https://marketingplatform.google.com/about/analytics/?utm_source=google-growth&utm_medium=referral-internal&utm_campaign=2018-q4-amer-all-gafree-analytics&utm_content=ads-